In [1]:
import multiprocessing
import contextlib
import time
from contextlib import contextmanager
import sys

In [23]:
import numpy as np
import tqdm

GLOBAL_START_TIME = time.time()


@contextmanager
def timer(name, silent=False):
    t0 = time.time()
    if not silent:
        print(
            "[% 8ds] %s" % (t0 - GLOBAL_START_TIME, name),
            flush=True,
            file=sys.stderr,
        )
    yield
    t1 = time.time()
    if not silent:
        print(
            "[% 8ds] %s done (%f seconds)" % (
                t1 - GLOBAL_START_TIME,
                name,
                t1 - t0
            ),
            flush=True,
            file=sys.stderr,
        )


def cut_nones(presults, mresults):
    """Cut entries that are None in a pair of lists. Any entry that is None
    in either list will exclude the item in the other.

    Parameters
    ----------
    presults : list
        One the list of things.
    mresults : list
        The other list of things.

    Returns
    -------
    pcut : list
        The cut list.
    mcut : list
        The cut list.
    """
    prr_keep = []
    mrr_keep = []
    for pr, mr in zip(presults, mresults):
        if pr is None or mr is None:
            continue
        prr_keep.append(pr)
        mrr_keep.append(mr)

    return prr_keep, mrr_keep


def _run_boostrap(x1, y1, x2, y2, wgts, silent):
    rng = np.random.RandomState(seed=100)
    mvals = []
    cvals = []
    if silent:
        itrl = range(500)
    else:
        itrl = tqdm.trange(
            500, leave=False, desc='running bootstrap', ncols=79,
            file=sys.stderr,
        )
    for _ in itrl:
        ind = rng.choice(len(y1), replace=True, size=len(y1))
        _wgts = wgts[ind].copy()
        _wgts /= np.sum(_wgts)
        mvals.append(np.mean(y1[ind] * _wgts) / np.mean(x1[ind] * _wgts) - 1)
        cvals.append(np.mean(y2[ind] * _wgts) / np.mean(x2[ind] * _wgts))

    return (
        np.mean(y1 * wgts) / np.mean(x1 * wgts), np.std(mvals),
        np.mean(y2 * wgts) / np.mean(x2 * wgts), np.std(cvals))


def _run_jackknife(x1, y1, x2, y2, wgts, jackknife):
    n_per = x1.shape[0] // jackknife
    n = n_per * jackknife
    x1j = np.zeros(jackknife)
    y1j = np.zeros(jackknife)
    x2j = np.zeros(jackknife)
    y2j = np.zeros(jackknife)
    wgtsj = np.zeros(jackknife)

    loc = 0
    for i in range(jackknife):
        wgtsj[i] = np.sum(wgts[loc:loc+n_per])
        x1j[i] = np.sum(x1[loc:loc+n_per] * wgts[loc:loc+n_per]) / wgtsj[i]
        y1j[i] = np.sum(y1[loc:loc+n_per] * wgts[loc:loc+n_per]) / wgtsj[i]
        x2j[i] = np.sum(x2[loc:loc+n_per] * wgts[loc:loc+n_per]) / wgtsj[i]
        y2j[i] = np.sum(y2[loc:loc+n_per] * wgts[loc:loc+n_per]) / wgtsj[i]

        loc += n_per

    # weighted jackknife from Busing et al. 1999, Statistics and Computing, 9, 3-8
    mhat = np.mean(y1[:n] * wgts[:n]) / np.mean(x1[:n] * wgts[:n])
    chat = np.mean(y2[:n] * wgts[:n]) / np.mean(x2[:n] * wgts[:n])
    mhatj = np.zeros(jackknife)
    chatj = np.zeros(jackknife)
    for i in range(jackknife):
        _wgts = np.delete(wgtsj, i)
        mhatj[i] = (
            np.sum(np.delete(y1j, i) * _wgts) / np.sum(np.delete(x1j, i) * _wgts)
        )
        chatj[i] = (
            np.sum(np.delete(y2j, i) * _wgts) / np.sum(np.delete(x2j, i) * _wgts)
        )

    tot_wgt = np.sum(wgtsj)
    mbar = jackknife * mhat - np.sum((1.0 - wgtsj/tot_wgt) * mhatj)
    cbar = jackknife * chat - np.sum((1.0 - wgtsj/tot_wgt) * chatj)

    hj = tot_wgt / wgtsj
    mtildej = hj * mhat - (hj - 1) * mhatj
    ctildej = hj * chat - (hj - 1) * chatj

    mvarj = np.sum((mtildej - mbar)**2 / (hj-1)) / jackknife
    cvarj = np.sum((ctildej - cbar)**2 / (hj-1)) / jackknife

    return (
        mbar,
        np.sqrt(mvarj),
        cbar,
        np.sqrt(cvarj),
    )


def estimate_m_and_c(
    presults,
    mresults,
    g_true,
    swap12=False,
    step=0.01,
    weights=None,
    jackknife=None,
    silent=False,
):
    """Estimate m and c from paired lensing simulations.

    Parameters
    ----------
    presults : list of iterables or np.ndarray
        A list of iterables, each with g1p, g1m, g1, g2p, g2m, g2
        from running metadetect with a `g1` shear in the 1-component and
        0 true shear in the 2-component. If an array, it should have the named
        columns.
    mresults : list of iterables or np.ndarray
        A list of iterables, each with g1p, g1m, g1, g2p, g2m, g2
        from running metadetect with a -`g1` shear in the 1-component and
        0 true shear in the 2-component. If an array, it should have the named
        columns.
    g_true : float
        The true value of the shear on the 1-axis in the simulation. The other
        axis is assumd to havea true value of zero.
    swap12 : bool, optional
        If True, swap the roles of the 1- and 2-axes in the computation.
    step : float, optional
        The step used in metadetect for estimating the response. Default is
        0.01.
    weights : list of weights, optional
        Weights to apply to each sample. Will be normalized if not already.
    jackknife : int, optional
        The number of jackknife sections to use for error estimation. Default of
        None will do no jackknife and default to bootstrap error bars.
    silent : bool, optional
        If True, do not print to stderr/stdout.

    Returns
    -------
    m : float
        Estimate of the multiplicative bias.
    merr : float
        Estimat of the 1-sigma standard error in `m`.
    c : float
        Estimate of the additive bias.
    cerr : float
        Estimate of the 1-sigma standard error in `c`.
    """

    with timer("prepping data for m,c measurement", silent=silent):
        if isinstance(presults, list) or isinstance(mresults, list):
            prr_keep, mrr_keep = cut_nones(presults, mresults)

            def _get_stuff(rr):
                _a = np.vstack(rr)
                g1p = _a[:, 0]
                g1m = _a[:, 1]
                g1 = _a[:, 2]
                g2p = _a[:, 3]
                g2m = _a[:, 4]
                g2 = _a[:, 5]

                if swap12:
                    g1p, g1m, g1, g2p, g2m, g2 = g2p, g2m, g2, g1p, g1m, g1

                return (
                    g1, (g1p - g1m) / 2 / step,
                    g2, (g2p - g2m) / 2 / step)

            g1p, R11p, g2p, R22p = _get_stuff(prr_keep)
            g1m, R11m, g2m, R22m = _get_stuff(mrr_keep)
        else:
            if swap12:
                g1p = presults["g2"]
                R11p = (presults["g2p"] - presults["g2m"]) / 2 / step
                g2p = presults["g1"]
                R22p = (presults["g1p"] - presults["g1m"]) / 2 / step

                g1m = mresults["g2"]
                R11m = (mresults["g2p"] - mresults["g2m"]) / 2 / step
                g2m = mresults["g1"]
                R22m = (mresults["g1p"] - mresults["g1m"]) / 2 / step
            else:
                g1p = presults["g1"]
                R11p = (presults["g1p"] - presults["g1m"]) / 2 / step
                g2p = presults["g2"]
                R22p = (presults["g2p"] - presults["g2m"]) / 2 / step

                g1m = mresults["g1"]
                R11m = (mresults["g1p"] - mresults["g1m"]) / 2 / step
                g2m = mresults["g2"]
                R22m = (mresults["g2p"] - mresults["g2m"]) / 2 / step

        if weights is not None:
            wgts = np.array(weights).astype(np.float64)
        else:
            wgts = np.ones(len(g1p)).astype(np.float64)
        wgts /= np.sum(wgts)

        msk = (
            np.isfinite(g1p) &
            np.isfinite(R11p) &
            np.isfinite(g1m) &
            np.isfinite(R11m) &
            np.isfinite(g2p) &
            np.isfinite(R22p) &
            np.isfinite(g2m) &
            np.isfinite(R22m))
        g1p = g1p[msk]
        R11p = R11p[msk]
        g1m = g1m[msk]
        R11m = R11m[msk]
        g2p = g2p[msk]
        R22p = R22p[msk]
        g2m = g2m[msk]
        R22m = R22m[msk]
        wgts = wgts[msk]

        y1 = (R11p + R11m)/2
        x1 = np.ones_like((g1p - g1m) / 2)

        y2 = (R22p + R22m)/2
        x2 = np.ones_like((g2p + g2m) / 2)

    if jackknife:
        with timer("running jackknife", silent=silent):
            return _run_jackknife(x1, y1, x2, y2, wgts, jackknife)
    else:
        with timer("running bootstrap", silent=silent):
            return _run_boostrap(x1, y1, x2, y2, wgts, silent)




In [30]:
import click
import fitsio
import esutil
import yaml
import numpy as np
import joblib

# from pizza_cutter_sims.run_utils import estimate_m_and_c, timer


def _read_ext_one(fname, ext):
    return fitsio.read(fname, ext=ext)


def _read_ext(files, ext):
    with joblib.Parallel(n_jobs=-1, backend='loky', max_nbytes=0, verbose=5) as par:
        data = par([
            joblib.delayed(_read_ext_one)(fname, ext)
            for fname in files
        ])

    with timer("combining arrays"):
        data = esutil.numpy_util.combine_arrlist(data)
    return data


def main(config, files, s2n_cuts, jackknife, ormask_cut, mfrac_cuts):
    """Measure shear from a set of pizza cutter sim FILES."""
    with open(config, "r") as fp:
        cfg = yaml.safe_load(fp.read())

    with timer("read plus data"):
        pdata = _read_ext(files, "plus")
    with timer("read minus data"):
        mdata = _read_ext(files, "minus")

    if mfrac_cuts is None:
        mfrac_cuts = [None]
    else:
        mfrac_cuts = [int(s) for s in mfrac_cuts.split(",")]

    for s2n_cut in [int(s) for s in s2n_cuts.split(",")]:
        for mfrac_cut in mfrac_cuts:
            with timer(
                "making cuts (s2n>={s2n_cut}, ormask: {ormask_cut}, "
                "mfrac: {mfrac_cut})".format(
                    s2n_cut=s2n_cut,
                    ormask_cut=ormask_cut,
                    mfrac_cut=mfrac_cut,
                )
            ):
                pmsk = (pdata["s2n_cut"] == s2n_cut)
                if ormask_cut is not None:
                    pmsk = pmsk & (pdata["ormask_cut"] == ormask_cut)
                else:
                    pmsk = pmsk & (pdata["ormask_cut"] == -1)
                if mfrac_cut is not None:
                    pmsk = pmsk & (pdata["mfrac_cut"] == mfrac_cut)
                else:
                    pmsk = pmsk & (pdata["mfrac_cut"] == -1)

                n_sims_msk = np.sum(pdata["weight"][pmsk])
                if n_sims_msk <= 0:
                    raise RuntimeError("Cuts did not keep any sims!")

            m, msd, c, csd = estimate_m_and_c(
                pdata[pmsk],
                mdata[pmsk],
                cfg["shear"]["g"],
                swap12=cfg["shear"]["swap12"],
                jackknife=jackknife,
                weights=pdata["weight"][pmsk],
            )

            print("""\
cuts: s2n>={s2n_cut}, ormask: {ormask_cut}, mfrac: {mfrac_cut}
    # of sims: {n_sims}
    R11   : {m: f} +/- {msd: f} [3-sigma]
    R22   : {c: f} +/- {csd: f} [3-sigma]""".format(
                    n_sims=n_sims_msk,
                    m=m,
                    msd=msd * 3,
                    c=c,
                    csd=csd * 3,
                    s2n_cut=s2n_cut,
                    ormask_cut=ormask_cut,
                    mfrac_cut=mfrac_cut,
                ),
                flush=True,
            )

In [42]:
import glob

pth = "/astro/u/beckermr/workarea/pizza-cutter-sims/runs"
run = "run0058_wcs-None_gals-dr_msk-None_coadd-None_mdet-g"
config = f"{pth}/{run}/config.yaml"
files = glob.glob(f"{pth}/{run}/*_nsims100000_seed1003565796.fits")
s2n_cuts = "10"
mfrac_cuts = "100"
jackknife = 200
ormask_cut = None

main(config, files, s2n_cuts, jackknife, ormask_cut, mfrac_cuts)

[     357s] read plus data
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
[     357s] combining arrays
[     357s] combining arrays done (0.001165 seconds)
[     357s] read plus data done (0.025519 seconds)
[     357s] read minus data
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
[     358s] combining arrays
[     358s] combining arrays done (0.000832 seconds)
[     358s] read minus data done (0.019526 seconds)
[     358s] making cuts (s2n>=10, ormask: None, mfrac: 100)
[     358s] making cuts (s2n>=10, ormask: None, mfrac: 100) done (0.001728 seconds)
[     358s] prepping data for m,c measurement
[     358s] prepping data for m,c measurement done (0.001404 seconds)
[     358s] running jackknife
[     358s] running jackknife done (0.021461 seconds)


cuts: s2n>=10, ormask: None, mfrac: 100
    # of sims: 100000.0
    R11   :  0.715277 +/-  0.007384 [3-sigma]
    R22   :  0.720284 +/-  0.009997 [3-sigma]


In [43]:
pth = "/astro/u/beckermr/workarea/pizza-cutter-sims/runs"
run = "run0064_wcs-None_gals-dr_msk-None_coadd-None_mdet-g_bkg-c"
config = f"{pth}/{run}/config.yaml"
files = glob.glob(f"{pth}/{run}/*_nsims100000_*.fits")
s2n_cuts = "10"
mfrac_cuts = "100"
jackknife = 200
ormask_cut = None

main(config, files, s2n_cuts, jackknife, ormask_cut, mfrac_cuts)

[     358s] read plus data
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
[     358s] combining arrays
[     358s] combining arrays done (0.000809 seconds)
[     358s] read plus data done (0.027169 seconds)
[     358s] read minus data
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
[     358s] combining arrays
[     358s] combining arrays done (0.000825 seconds)
[     358s] read minus data done (0.018332 seconds)
[     358s] making cuts (s2n>=10, ormask: None, mfrac: 100)
[     358s] making cuts (s2n>=10, ormask: None, mfrac: 100) done (0.002362 seconds)
[     358s] prepping data for m,c measurement
[     358s] prepping data for m,c measurement done (0.001363 seconds)
[     358s] running jackknife
[     358s] running jackknife done (0.020792 seconds)


cuts: s2n>=10, ormask: None, mfrac: 100
    # of sims: 99800.0
    R11   :  0.835688 +/-  0.009752 [3-sigma]
    R22   :  0.829776 +/-  0.011989 [3-sigma]
